## Clean

In [33]:
import pandas as pd


for i in ["train.csv", "test.csv", "sample_test.csv"]:
    df = pd.read_csv("../dataset/" + i)
    df["image_link"] = df["image_link"].str.rstrip("/")
    df["image_link"] = df["image_link"].apply(lambda x: x + ".jpg" if x[-4:] != ".jpg" else x)
    df.to_csv("dataset/" + i, index=False)

## Download

In [34]:
import pandas as pd
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
from io import BytesIO

df = pd.read_csv('../dataset/train.csv')

count = {'height': 0, 'width': 0, 'depth': 0, 'item_volume': 0, 'item_weight': 0, 'maximum_weight_recommendation': 0, 'wattage': 0, 'voltage': 0}
required = {'height': 100, 'width': 100, 'depth': 100, 'item_volume': 300, 'item_weight': 150, 'maximum_weight_recommendation': 150, 'wattage': 300, 'voltage': 300}
append = []
for i, row in df.iterrows():
    if count[row['entity_name']] < required[row['entity_name']]:
        count[row['entity_name']] += 1
        append.append(row)

df = pd.DataFrame(append)

In [37]:
def download(url):
    filename = url.split('/')[-1]
    filepath = os.path.join('../images', filename)
    if not os.path.exists(filepath):
        try:
            response = requests.get(url)
            image = Image.open(BytesIO(response.content))
            image.thumbnail((1080, 1080), Image.LANCZOS)
            new_image = Image.new("RGB", (1080, 1080), "white")
            new_image.paste(image, ((new_image.width - image.width) // 2, (new_image.height - image.height) // 2))
            new_image.save(filepath)
        except Exception as e:
            print(f"Skipped image: {url} with error: {e}")
    return filepath

os.makedirs('../images', exist_ok=True)

with ThreadPoolExecutor(max_workers=10) as executor:
    image_paths = list(executor.map(download, df['image_link']))

Skipped image: https://m.media-amazon.com/images/I/1yw53vfQtS.jpg with error: cannot identify image file <_io.BytesIO object at 0x0000014923674B30>
Skipped image: https://m.media-amazon.com/images/I/41NH8WgeBOL.jpg with error: image file is truncated (2 bytes not processed)
